In [1]:
from tifffile import imread
from pathlib import Path

In [2]:
img_files = sorted(Path(r'Y:\Eric\prediction_test\predictions').glob('*.tif'))
img_files

[WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000001_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000002_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000003_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000004_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000005_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000006_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000007_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000008_Nz54P.tif'),
 WindowsPath('Y:/Eric/prediction_test/predictions/kdv1502R_5L_30ms_300gain002_pos5_ch1_frame000009_Nz54P

In [3]:
from tifffile import imsave
import os
from skimage.measure import regionprops
import numpy as np

for i, file_name in enumerate(img_files):
    im = imread(str(file_name))
    props = regionprops(im)

    
    Tracking_stack = np.zeros((np.asarray(im.shape) / 2).astype(np.int16))
    
    centroids = np.asarray([p.centroid for p in props])
    centroids = centroids / 2
    
    
    centroids = np.rint(centroids)
    centroids = centroids.astype(np.int16)
    
    np.save(f'centroids{i:04d}', centroids)
    
    #for z, y, x in centroids:
    #    Tracking_stack[z, y, x] = 1

    #imsave(os.path.join(f'track{i:04d}.tif'), Tracking_stack)

In [9]:
Tracking_stack.shape

(108, 512, 512)

In [5]:
centroid_files = sorted(Path('Y:\Eric\prediction_test').glob('*.npy'))

In [6]:
centroid_files

[WindowsPath('Y:/Eric/prediction_test/centroids0000.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0001.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0002.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0003.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0004.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0005.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0006.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0007.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0008.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0009.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0010.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0011.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0012.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0013.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0014.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0015.npy'),
 WindowsPath('Y:/Eric/prediction_test/centroids0016.npy'

In [10]:
shape = Tracking_stack.shape

In [13]:
for i, p in enumerate(centroid_files):
    centroid = np.load(str(p))
    
    Tracking_stack = np.zeros(shape, dtype=np.uint8)
    
    for z, y, x in centroid:
        Tracking_stack[z, y, x] = 255

    imsave(os.path.join('T:', f'track{i:04d}.tif'), Tracking_stack)

In [25]:
Tracking_stack = np.zeros((shape[0]*len(centroid_files), *shape[1:]), dtype=np.int8)

for i, p in enumerate(centroid_files):
    centroid = np.load(str(p))
 
    for z, y, x in centroid:
        Tracking_stack[z+shape[0]*i, y, x] = 255

imsave(os.path.join('T:', f'tracks_all.tif'), Tracking_stack)

In [18]:
len(centroid_files)

99

In [22]:
Tracking_stack_ = np.reshape(Tracking_stack, (99, 108, 512, 512))

In [24]:
np.where(Tracking_stack_)

(array([ 0,  0,  0, ..., 98, 98, 98], dtype=int64),
 array([10, 10, 10, ..., 26, 29, 37], dtype=int64),
 array([237, 248, 262, ..., 234, 374, 268], dtype=int64),
 array([242, 266, 275, ..., 230, 222, 301], dtype=int64))

In [14]:
im = imread(r"T:\tracks\track0091.tif")

In [15]:
np.unique(im)

array([0, 1], dtype=uint8)

In [51]:
from tifffile import imsave

Tracking_stack = np.zeros((*im.shape[1:], len(img_files)), dtype=np.uint8)

for i, file_name in enumerate(img_files):
    im = imread(str(file_name))
    props = regionprops(im)
    centroids = np.asarray([p.centroid for p in props])
    centroids = np.rint(centroids[:, 1:])
    centroids = centroids.astype(np.int16)
      
    
    for y, x in centroids:
        Tracking_stack[y, x, i] = 1

imsave(f'track_2d.tif', Tracking_stack)

In [53]:
tracking_stack = np.swapaxes(Tracking_stack, -1, 0)

In [55]:
imsave(f'track_2d.tif', tracking_stack)